In [2]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

from PIL import Image
import skimage

import sys, os

import geojson
import json

from IPython.display import clear_output

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

In [3]:
import planet

from planet import api
from planet.api import filters
from sys import stdout

In [6]:
myPlanetKey = "664df19c4dcd42b3b7d14acd5b9936b4"

In [7]:
# will pick up api_key via environment variable PL_API_KEY
# but can be specified using `api_key` named argument
client = api.ClientV1(api_key=myPlanetKey)

### Search by AOI, time, and other properties (cloud cover)

In [8]:
# cloudy by date - a silly example to see when it is cloudiest according
# to planet imagery
#
# output a CSV of PlanetScope item-id, cloud cover, and date acquired
# for a region around the San Fransisco peninsula

# geojson AOI
aoi = {
  "type": "Polygon",
  "coordinates": [
    [
      [-122.54, 37.81],
      [-122.38, 37.84],
      [-122.35, 37.71],
      [-122.53, 37.70],
      [-122.54, 37.81]
    ]
  ]
}

In [10]:
# build a query using the AOI and
# a cloud_cover filter that excludes 'cloud free' scenes
query = filters.and_filter(
    filters.geom_filter(aoi),
    filters.range_filter('cloud_cover', gt=0),
)

query

{'config': ({'config': {'coordinates': [[[-122.54, 37.81],
      [-122.38, 37.84],
      [-122.35, 37.71],
      [-122.53, 37.7],
      [-122.54, 37.81]]],
    'type': 'Polygon'},
   'field_name': 'geometry',
   'type': 'GeometryFilter'},
  {'config': {'gt': 0}, 'field_name': 'cloud_cover', 'type': 'RangeFilter'}),
 'type': 'AndFilter'}

In [21]:
# build a request for only PlanetScope imagery
request = filters.build_search_request(
    query, item_types=['PSScene4Band']
)

# if you don't have an API key configured, this will raise an exception
result = client.quick_search(request)

stdout.write('id,cloud_cover,date\n')

# items_iter returns a limited iterator of all results. behind the scenes,
# the client is paging responses from the API
for item in result.items_iter(limit=None):
    props = item['properties']
    stdout.write('{0},{cloud_cover},{acquired}\n'.format(item['id'], **props))

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


id,cloud_cover,date
20170419_180645_1010,0.07,2017-04-19T18:06:45.464541Z
20170419_180644_1010,0.27,2017-04-19T18:06:44.407174Z
20170419_181313_0e0d,0.03,2017-04-19T18:13:13.697499Z
20170419_180521_1009,0.09,2017-04-19T18:05:21.772777Z
20170419_180520_1009,0.01,2017-04-19T18:05:20.715291Z
20170419_180519_1009,0.03,2017-04-19T18:05:19.657805Z
20170419_180518_1009,0.01,2017-04-19T18:05:18.600319Z
20170419_181316_0e0d,0.1,2017-04-19T18:13:16.913472Z
20170419_181315_0e0d,0.02,2017-04-19T18:13:15.841481Z
20170419_181314_0e0d,0.03,2017-04-19T18:13:14.76949Z
20170418_180653_1017,0.69,2017-04-18T18:06:53.053834Z
20170418_180651_1017,0.72,2017-04-18T18:06:51.9988Z
20170415_180505_1024,0.01,2017-04-15T18:05:05.023325Z
20170415_180503_1024,0.22,2017-04-15T18:05:03.956408Z
20170415_180502_1024,0.21,2017-04-15T18:05:02.889491Z
20170415_180501_1024,0.04,2017-04-15T18:05:01.822574Z
20160515_153305_1_0c70,0.03,2016-05-15T15:33:05.787533Z
20170415_181354_0e16,0.04,2017-04-15T18:13:54.915159Z
20170415_1

### Download imagery

In [26]:
from planet.api import downloader

In [53]:
item['_links']

{u'_self': u'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20160914_180659_0e0d',
 u'assets': u'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20160914_180659_0e0d/assets/',
 u'thumbnail': u'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20160914_180659_0e0d/thumb'}

In [44]:
item.keys()

[u'geometry', u'id', u'_links', u'_permissions', u'type', u'properties']

In [27]:
from planet.api import downloader
from planet.api.utils import handle_interrupt
import logging
import sys
from concurrent.futures import Future
import threading
import time

In [28]:
SPEED_UP = 1000.
WRITE_DELAY = 3 / SPEED_UP
ACTIVATE_DELAY = .5 / SPEED_UP
DOWNLOAD_DELAY = .5 / SPEED_UP
ASSET_DELAY = .5 / SPEED_UP

class Resp(object):

    def __init__(self, resp):
        self.resp = resp

    def get(self):
        return self.resp


class Body(object):
    def __init__(self, name):
        self.name = name
        self._got_write = False

    def write(self, file, callback):
        callback(start=self)
        callback(total=1024, wrote=1024)
        callback(finish=self)
        self._got_write = True

    def await(self):
        pass

    def cancel(self):
        pass


class Download(object):
    # mirror models.Response kinda, mostly not
    def __init__(self, body, writer):
        self._future = Future()

        def respond():
            self._future.set_result(body)
            writer(body)
        # don't write to the body synchronously
        threading.Timer(WRITE_DELAY, respond).start()

    def await(self):
        return self._future.result()


def asset(name, type, status):
    return {'_name': name, 'type': type, 'status': status,
            'location': 'http://somewhere/%s/%s' % (type, name)}


def assets(*assets):
    res = {}
    for a in assets:
        res[a['type']] = a
    res['_pinged'] = 0
    return res


class HelperClient(object):

    def __init__(self):
        self.assets = {}
        self._shutdown = False

    def get_assets(self, item):
        if item['id'] in self.assets:
            a = self.assets[item['id']]
            if a['_pinged'] > 2:
                a['a']['status'] = 'active'
                a['b']['status'] = 'active'
            a['_pinged'] += 1
        else:
            a = assets(asset(item['id'] + '.junk', 'a', 'inactive'),
                       asset(item['id'] + '.crud', 'b', 'inactive'))
            self.assets[item['id']] = a
        time.sleep(ASSET_DELAY)
        return Resp(a)

    def activate(self, asset):
        time.sleep(ACTIVATE_DELAY)
        asset['status'] = 'activating'

    def download(self, asset, writer):
        time.sleep(DOWNLOAD_DELAY)
        b = Body(asset['_name'])
        return Download(b, writer)

    def shutdown(self):
        self._shutdown = True


def items_iter(cnt):
    for i in range(cnt):
        yield {'id': str(i)}


In [55]:
logging.basicConfig(
    stream=sys.stderr, level=logging.INFO,
    format='%(asctime)s %(message)s', datefmt='%M:%S'
)
cl = HelperClient()
items = items_iter(3)
asset_types = ['analytic', 'visual']
dl = downloader.create(
    cl, no_sleep=True,
    astage__size=10, pstage__size=10, pstage__min_poll_interval=0,
    dstage__size=2)
completed = []
dl.on_complete = lambda *a: completed.append(a)
stats = handle_interrupt(dl.shutdown, dl.download, items,
                         asset_types, 'dest')
assert stats == {
    'downloading': 0, 'complete': 200, 'paging': False,
    'downloaded': '0.20MB', 'activating': 0, 'pending': 0
}
assert 200 == len(completed)



32:53 no desired assets in item, skipping
32:53 no desired assets in item, skipping
32:53 no desired assets in item, skipping


AssertionError: 

In [35]:
dl.download()

{'activating': 0,
 'complete': 200,
 'downloaded': '0.20MB',
 'downloading': 0,
 'paging': False,
 'pending': 0}